In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Connect to db
conn = sqlite3.connect('data/8anu.sqlite')

# Read data into pandas
sql = ('SELECT a.id, a.user_id, a.grade_id, a.notes, a.raw_notes, a.method_id, '
       'a.date, a.year, a.name, a.crag_id, a.crag, a.sector_id, a.sector, a.country AS crag_country, a.rating, '
       'g.fra_routes, m.shorthand AS method, '
       'u.city, u.country AS user_country, u.sex, u.height, u.weight, u.started, u.birth '
       'FROM ('
           'SELECT * '
           'FROM ascent '
           'WHERE crag=\'Frankenjura\' AND climb_type=0' 
       ') AS a '
       'LEFT OUTER JOIN grade AS g ON a.grade_id=g.id '
       'LEFT OUTER JOIN method AS m ON a.method_id=m.id '
       'LEFT OUTER JOIN user AS u ON a.user_id=u.id '
       )
df = pd.read_sql_query(sql, conn)

# Close db connection
conn.close()

`date` seconds since 1970-01-01

In [3]:
df.describe()

,id,user_id,grade_id,raw_notes,method_id,date,year,crag_id,sector_id,rating,sex,height,weight,started
count,1.145890e+05,114589.000000,114589.000000,114589.000000,114589.000000,1.145890e+05,114589.000000,114589.0,114589.000000,114589.000000,114587.000000,114587.000000,114587.000000,114587.000000
mean,2.793723e+06,28385.817609,47.875023,1.519884,1.935273,1.324583e+09,2011.429238,16600.0,1304.176308,1.011615,0.162488,143.181766,47.486094,1627.095866
std,1.369897e+06,17688.155664,8.862341,4.114874,0.948231,1.404149e+08,9.458719,0.0,1999.530178,1.189323,0.368899,69.778278,33.250039,778.451573
min,5.820000e+02,3.000000,7.000000,0.000000,1.000000,0.000000e+00,13.000000,16600.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.693286e+06,12830.000000,42.000000,0.000000,1.000000,1.246313e+09,2009.000000,16600.0,156.000000,0.000000,0.000000,165.000000,0.000000,1984.000000
50%,2.950451e+06,27965.000000,49.000000,0.000000,2.000000,1.347401e+09,2012.000000,16600.0,513.000000,0.000000,0.000000,175.000000,63.000000,1999.000000
75%,3.917280e+06,41517.000000,55.000000,0.000000,3.000000,1.433542e+09,2015.000000,16600.0,1783.000000,2.000000,0.000000,180.000000,73.000000,2005.000000
max,4.910168e+06,66965.000000,79.000000,80.000000,4.000000,1.505167e+09,2017.000000,16600.0,13027.000000,3.000000,1.000000,255.000000,100.000000,2090.000000


In [4]:
df.columns

Index(['id', 'user_id', 'grade_id', 'notes', 'raw_notes', 'method_id', 'date',
       'year', 'name', 'crag_id', 'crag', 'sector_id', 'sector',
       'crag_country', 'rating', 'fra_routes', 'method', 'city',
       'user_country', 'sex', 'height', 'weight', 'started', 'birth'],
      dtype='object')

In [7]:
# Export to .csv for submitting Project Proposal to Udacity
df.to_csv('data/8anu_franken.csv', sep=';')